In [1]:
import numpy as np
import json_tricks 

import glob
import os
import re

import pandas as pd

In [2]:
def load_all(file="*"):
    filenames = glob.glob('output/'+file+'.json')
    if len(filenames) == 0:
        raise Exception("No files match the given pattern.")

    p_experiment = re.compile('.*(experiment_[a-zA-Z0-9-]+)_')

    outputs = {}
    
    for filename in filenames:
        # First get experiment name
        m = re.match(p_experiment, filename)
        experiment = m.groups()[0] if m else 'default'
        if experiment not in outputs:
            outputs[experiment] = []

        i = len(outputs[experiment])
        full_filename = u'\\\\?\\' + os.path.abspath(filename)
        with open(full_filename) as fh:
            o = json_tricks.load(fh, preserve_order=False)
            
        outputs[experiment].append(o)

    return outputs

In [3]:
def create_CSV(name, suffix='', extra_param_names = []):
    outputs = load_all(name+'_*')
    all_df = [pd.DataFrame({
        'name': outp['meta']['shorttitle'],
        'year': outp['meta']['t_values_years'],
        'p': outp['p'],
        'E': outp['E'],
        'baseline': outp['baseline'],
        'Y': outp['Y'],
        'Ygross': outp['Ygross'],
        'damageFraction': outp['damageFraction'],
        'abatementFraction': (outp['Y'] - (outp['investments'] + outp['consumption'])) / outp['Ygross'],
        'CE': outp['cumEmissions'],
        'temp': outp['temp'],
        'SSP': outp['meta']['params'].default_params['SSP'],
        'damage': outp['meta']['params'].default_params['damage'],
        'TCRE': outp['meta']['params'].default_params['TCRE'],
        'cost_level': outp['meta']['params'].default_params['cost_level'],
        'r': outp['meta']['params'].default_params['r'],
        'gamma': outp['meta']['params'].default_params['gamma'],
        'beta': outp['meta']['params'].default_params['beta'],
        'elasmu': outp['meta']['params'].default_params['elasmu'],
        'on_utility': outp['meta']['params'].default_params['maximise_utility'] if 'maximise_utility' in outp['meta']['params'].default_params else np.nan,
        **{name: (outp['meta']['params'].default_params[name] if name in outp['meta']['params'].default_params else '') for name in extra_param_names}
    }) for outp in outputs[name]]

    df = pd.concat(all_df)

    df['p_rel'] = df['p'] / df['gamma']
    
    df.to_csv('{}{}.csv'.format(name,suffix), index=False)
    

In [5]:
%%time
create_CSV("experiment_all", extra_param_names=['carbonbudget', 'useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 58.8 s


In [7]:
%%time
create_CSV("experiment_alldrupp", extra_param_names=['carbonbudget', 'useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 11.4 s


In [6]:
%%time
create_CSV('experiment_allcba', extra_param_names=['useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 52.2 s


In [7]:
%%time
create_CSV('experiment_allcbadrupp', extra_param_names=['useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

C:\Users\wijstvdk\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


Wall time: 3.49 s


In [6]:
%%time
create_CSV("experiment_extra", extra_param_names=['carbonbudget', 'useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 237 ms


In [5]:
%%time
create_CSV("experiment_cbaminemissions", extra_param_names=['carbonbudget', 'useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 1.27 s


In [16]:
%%time
create_CSV('experiment_extracba', extra_param_names=['useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 157 ms


In [19]:
%%time
create_CSV('experiment_drupp', extra_param_names=['useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 713 ms


In [21]:
%%time
create_CSV('experiment_druppthree', extra_param_names=['useBaselineCO2Intensity', 'maxReductParam', 'minEmissions'])

Wall time: 61.8 ms
